In [1]:
!pip install tensorflow pillow opencv-python
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import cv2
from tensorflow.keras.preprocessing import image
from PIL import Image
import matplotlib.pyplot as plt

    pytz>=2011n
        ~~~~~~^


In [2]:
tf.__version__

'2.12.0'

In [3]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)
training_set = train_datagen.flow_from_directory('training set',
                                                 target_size=(64, 64),
                                                 batch_size=7,
                                                 class_mode='categorical')

Found 1137 images belonging to 10 classes.


In [4]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_set = test_datagen.flow_from_directory('test-set',
                                            target_size=(64, 64),
                                            batch_size=7,
                                            class_mode='categorical')


Found 295 images belonging to 10 classes.


In [5]:
# Building the CNN
cnn = tf.keras.models.Sequential()

# Convolution
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

# Pooling
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# Second convolutional layer
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# Flattening
cnn.add(tf.keras.layers.Flatten())

# Full connection
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

# Output layer - Updated for 10 classes
cnn.add(tf.keras.layers.Dense(units=10, activation='softmax'))

# Compiling CNN
cnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Training and evaluating the CNN
cnn.fit(x=training_set, validation_data=test_set, epochs=8)

Epoch 1/8
163/163 [==============================] - 13s 79ms/step - loss: 2.1573 - accuracy: 0.2120 - val_loss: 2.0898 - val_accuracy: 0.2644
Epoch 2/8
163/163 [==============================] - 3s 19ms/step - loss: 1.7890 - accuracy: 0.3624 - val_loss: 1.5661 - val_accuracy: 0.4237
Epoch 3/8
163/163 [==============================] - 3s 19ms/step - loss: 1.4946 - accuracy: 0.4732 - val_loss: 1.4771 - val_accuracy: 0.5085
Epoch 4/8
163/163 [==============================] - 3s 19ms/step - loss: 1.3282 - accuracy: 0.5356 - val_loss: 1.4448 - val_accuracy: 0.5119
Epoch 5/8
163/163 [==============================] - 3s 19ms/step - loss: 1.1825 - accuracy: 0.5761 - val_loss: 1.2442 - val_accuracy: 0.5424
Epoch 6/8
163/163 [==============================] - 3s 19ms/step - loss: 1.0174 - accuracy: 0.6491 - val_loss: 1.0999 - val_accuracy: 0.6441
Epoch 7/8
163/163 [==============================] - 3s 19ms/step - loss: 0.9202 - accuracy: 0.6763 - val_loss: 1.3601 - val_accuracy: 0.5559
Epoch

In [6]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [7]:

cap = cv2.VideoCapture(0) 

In [8]:
class_labels = list(training_set.class_indices.keys())


confidence_threshold = 0.5

In [10]:

confidence_threshold = 0.4

while True:
 
    ret, frame = cap.read()
    
    gray_image = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    faces = face_cascade.detectMultiScale(gray_image, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    
    for (x, y, w, h) in faces:

        face = frame[y:y+h, x:x+w]
        
        face_img = cv2.resize(face, (64, 64))
        face_img = image.img_to_array(face_img)
        face_img = np.expand_dims(face_img, axis=0)
        face_img = face_img / 255.0  
        
        result = cnn.predict(face_img)
        predicted_class_index = np.argmax(result)
        prediction_confidence = np.max(result)  
        
        if prediction_confidence >= confidence_threshold:
            prediction = class_labels[predicted_class_index]
        else:
            prediction = "New face detected"
        
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(frame, prediction, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    
    cv2.imshow('Face Recognition', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


1/1 [==============================] - 0s 15ms/step


: 

In [ ]:
cap.release()
cv2.destroyAllWindows()